In [227]:
from qoop.core import state, ansatz, metric
import qiskit
from qiskit import transpile
from qiskit.quantum_info import Operator, DensityMatrix, Kraus
from scipy.linalg import qr
import numpy as np

<img src = './docs/1.png' height ='800px'>

In [228]:
# Step 1: Read quantum compilation and learn how to use qoop
# https://github.com/vutuanhai237/qoop/wiki/Advance:-Custom-state-preparation

# Step 2: Implement the following function

#Create a ansatz V with n qubits
def V(num_qubits: int):
    return ansatz.stargraph (num_qubits=num_qubits)

# def Epsilon(rho, kraus_operators):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return sum(K @ rho.data @ np.transpose(np.conjugate(K)) for K in kraus_operators)

# def Epsilon2(rho, unitary_matrix):
#     # K = K_noise = [\sqrt(p) I @ I, \sqrt(1-p) Z @ Z]
#     # see Eq. 1 Ref. [1]
#     return (np.transpose(np.conjugate(unitary_matrix)) @ rho.data @ unitary_matrix)

def calRho3 (rho, unitary_matrix, kraus_operators):
    rho2 = sum(K @ rho.data @ np.transpose(np.conjugate(K)) for K in kraus_operators)
    rho3 = (np.transpose(np.conjugate(unitary_matrix)) @ rho2.data @ unitary_matrix)
    return rho3

def createKraus(unitary_matrix):
    kraus_ops = []
    Q, R = qr(unitary_matrix) #(16,16) -> vector 1x16

    #print(Q.shape)
    for q in Q:
        
        q = np.expand_dims(q, 1)
        #print(q)
        #print(np.transpose(np.conjugate(q)))
        #print(q @ np.transpose(np.conjugate(q)))
        kraus_ops.append(q @ np.transpose(np.conjugate(q))) #vector 1x16

    
    return kraus_ops



In [229]:
num_qubits = 2

#Create a ansatz V with n qubits
circuit = V(num_qubits)

#Assign random parameter
num_params = circuit.num_parameters
x0 = 2 * np.pi * np.random.random(num_params)
circuit = circuit.assign_parameters(dict(zip(circuit.parameters, x0)))

print(circuit)

     ┌────────────┐   
q_0: ┤ Ry(5.1115) ├─■─
     ├────────────┤ │ 
q_1: ┤ Ry(4.0308) ├─■─
     └────────────┘   


In [230]:
# Get the unitary operator corresponding to the circuit
unitary_op = Operator(circuit)

# Get the unitary matrix
unitary_matrix = unitary_op.data
print (unitary_matrix.shape) #(16, 16 as num_qubits is 4)

(4, 4)


In [231]:
KrausOperators = createKraus(unitary_matrix=unitary_matrix)
print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperators))


[[ 1.00000000e+00+0.j -5.55111512e-17+0.j  1.24900090e-16+0.j
   8.32667268e-17+0.j]
 [-5.55111512e-17+0.j  1.00000000e+00+0.j  1.66533454e-16+0.j
  -1.11022302e-16+0.j]
 [ 1.24900090e-16+0.j  1.66533454e-16+0.j  1.00000000e+00+0.j
   1.66533454e-16+0.j]
 [ 8.32667268e-17+0.j -1.11022302e-16+0.j  1.66533454e-16+0.j
   1.00000000e+00+0.j]]


In [232]:
#rho
# rho = DensityMatrix.from_label('0' * 2)
# rho2 = Epsilon(rho=rho, kraus_operators=KrausOperators)
# print(rho2)

# rho3 = Epsilon2(rho=rho2, unitary_matrix=unitary_matrix)
# print(rho3)


In [233]:
#rho
rho = DensityMatrix.from_label('0' * num_qubits)
rho3 = calRho3(rho=rho, kraus_operators=KrausOperators, unitary_matrix=unitary_matrix)
print(rho3)


[[ 0.26896447+0.j  0.10549981+0.j  0.0591176 +0.j  0.15253954+0.j]
 [ 0.10549981+0.j  0.13468363+0.j  0.06494594+0.j  0.11828246+0.j]
 [ 0.0591176 +0.j  0.06494594+0.j  0.19489938+0.j -0.00831074+0.j]
 [ 0.15253954+0.j  0.11828246+0.j -0.00831074+0.j  0.40145251+0.j]]


In [234]:
#cost func compare rho, rho3
def cost(rho, rho3):
    return 1-metric.compilation_trace_fidelity(rho, rho3)

print(cost(rho=rho, rho3=rho3))

#Nhận U, K trả về rho3

0.5126054893430912


In [235]:
# Step 3: Implement the following function
# state_need_tomogaphy = ...
# rho = np.conjugate(np.transpose(state_need_tomogaphy)) @ state_need_tomogaphy   # density matrix
# rho' = Delta(rho)
# compiler = qoop.qcompilation.QuantumCompilation(U = rho', V = V())
# compiler.fit()
# compiler.plot()
# see fidelities versus iteration

<img src = './docs/2.png' height ='800px'>

<img src = './docs/3.jpg' height ='800px'>

In [422]:
def Derivative(rho, unitary_matrix, kraus_operators, epsilon=0.01, alpha=0.1):
    kraus_operators_plus = kraus_operators.copy()
    kraus_operators_minus = kraus_operators.copy()
    c = 1
    
    # Loop through Kraus operators
    for n, k in enumerate(kraus_operators):
        k_plus = k + epsilon # Kj(+epsilon)
        k_minus = k - epsilon # Kj(-epsilon)
        
        kraus_operators_plus[n] = k_plus # Replace Kj by new Kj_plus
        kraus_operators_minus[n] = k_minus # Replace Kj by new Kj_minus
        #print(kraus_operators_minus)
        #print(kraus_operators_plus)

        # Compute fidelity for K+ and K-
        fil_plus = metric.compilation_trace_fidelity(rho, calRho3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_plus))
        fil_minus = metric.compilation_trace_fidelity(rho, calRho3(rho=rho, unitary_matrix=unitary_matrix, kraus_operators=kraus_operators_minus))
        
        # Compute the derivative
        derivative = (fil_plus - fil_minus) / (2*epsilon)

        # Accumulate
        c *= derivative
        
        # Reset Kraus operators for next iteration
        kraus_operators_plus[n] = k
        kraus_operators_minus[n] = k

        
    
    # Update Kraus operators
    kraus_operators_res = [k - alpha * c for k in kraus_operators]
    print ('c=', c)
    #print(kraus_operators_res)
    return kraus_operators_res

In [ ]:
# Create a copy
KrausOperatorsTry = KrausOperators.copy()
a = 1

# Loop until the cost <= 0.01
while (a > 0.01):
    
    KrausOperatorsTry = Derivative(rho, unitary_matrix, KrausOperatorsTry) # Update Kraus Operators 
    print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry)) # Should be I though
    a = cost(rho, calRho3(rho, unitary_matrix, KrausOperatorsTry))
    print (a)

In [424]:
KrausOperatorsTry = KrausOperators.copy()

In [463]:
# try looping manually
print(sum(K @ np.transpose(np.conjugate(K)) for K in KrausOperatorsTry))
a = cost(rho, calRho3(rho, unitary_matrix, KrausOperatorsTry))
KrausOperatorsTry = Derivative(rho, unitary_matrix, KrausOperatorsTry) 
print (a)

[[1.1343327+0.j 0.1343327+0.j 0.1343327+0.j 0.1343327+0.j]
 [0.1343327+0.j 1.1343327+0.j 0.1343327+0.j 0.1343327+0.j]
 [0.1343327+0.j 0.1343327+0.j 1.1343327+0.j 0.1343327+0.j]
 [0.1343327+0.j 0.1343327+0.j 0.1343327+0.j 1.1343327+0.j]]
c= -0.0007357447130613236
0.5305235521980403
